In [114]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
#path = "D:\\Mes Documents\\Football data\\"
path = "E:\\nosave\\Football data\\"

#Import de la saison 2018-2019
data2223 = pd.read_csv(path +"season-2223.csv")
data2122 = pd.read_csv(path +"season-2122.csv")
data2021 = pd.read_csv(path +"season-2021.csv")
data1920 = pd.read_csv(path +"season-1920.csv")
data1819 = pd.read_csv(path +"season-1819.csv")
data1718 = pd.read_csv(path +"season-1718.csv")
data1617 = pd.read_csv(path +"season-1617.csv")
data1516 = pd.read_csv(path +"season-1516.csv")
data1415 = pd.read_csv(path +"season-1415.csv")
data1314 = pd.read_csv(path +"season-1314.csv")
data1213 = pd.read_csv(path +"season-1213.csv")
data1112 = pd.read_csv(path +"season-1112.csv")
data1011 = pd.read_csv(path +"season-1011.csv")
data0910 = pd.read_csv(path +"season-0910.csv")

#Combine dataset

seasons = [data1011,data1112,data1213,data1314,data1415,data1516,data1617,data1718,data1819,data1920,data2021,data2122,data2223]
columns = ["Date","HomeTeam","AwayTeam","FTR","FTHG","FTAG","HTHG","HTAG"]
endofseasondf = pd.DataFrame(columns=columns)
entry = pd.DataFrame.from_dict({
     "Date":["End of season"],
     "HomeTeam":[None],
     "AwayTeam":[None],
     "FTR":[None],
     "FTHG":[None],
     "FTAG":[None],
     "HTHG":[None],
     "HTAG":[None]
})
endofseasondf = pd.concat([endofseasondf, entry], ignore_index=True)

data0922 = data0910[columns]
for d in seasons:
    data0922 = pd.concat([data0922,endofseasondf,d[["Date","HomeTeam","AwayTeam","FTR","FTHG","FTAG","HTHG","HTAG"]]])

In [115]:
#Clean different name for a same team

data0922.replace("Ajaccio","AC Ajaccio",inplace=True)
data0922.replace("Ajaccio GFCO","AC Ajaccio",inplace=True)

In [116]:
#List of the teams
data = data0922

teams = list(data["HomeTeam"].drop_duplicates())
teams.append("Clermont")
print(f"There are {len(teams)} differents teams between 2009 and 2019, we are only interested in the actual 20 teams")

actual_teams = ["Paris SG",
            "Monaco",
            "Lens",
            "Montpellier",
            "Rennes",
            "Marseille",
            "Lorient",
            "Lille",
            "Lyon",
            "Nice",
            "Clermont",
            "Reims",
            "Toulouse",
            "Troyes",
            "Nantes",
            "Brest",
            "Auxerre",
            "AC Ajaccio",
            "Strasbourg",
            "Angers"]

There are 39 differents teams between 2009 and 2019, we are only interested in the actual 20 teams


### WDL streak

In [326]:
# Recalculate final ranking & parcours - Pour une saison - Utile pour check la validité des datas

scores = {team : 0 for team in teams}
parcours = {team : [] for team in teams}

for i in range (len(data)):
    match = data.iloc[i]
    home_team,away_team = match.HomeTeam,match.AwayTeam
    result = match.FTR
    if result == "H":
        scores[home_team]+=3
        parcours[home_team].append("W")
        parcours[away_team].append("L")
    elif result == "A":
        scores[away_team]+=3
        parcours[away_team].append("W")
        parcours[home_team].append("L")
    else:
        scores[away_team]+=1
        scores[home_team]+=1
        parcours[away_team].append("D")
        parcours[home_team].append("D")
    
scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1],reverse=True)}

final_ranking = pd.DataFrame()
final_ranking["Team"] = list(scores.keys())
final_ranking["Score"] = list(scores.values())

In [228]:
#Compute all saisons parcours

seasons  = {} #Will store the parcours of the actual teams for each seasons
parcours = {team : [] for team in teams}
count = 2009
for i in range (len(data)):
    match = data.iloc[i]
    if match.Date == "End of season":
        seasons[f"{count}"] = {key : parcours[key] for key in actual_teams}
        count+=1
        parcours = {team : [] for team in teams}
    home_team,away_team = match.HomeTeam,match.AwayTeam
    if home_team not in actual_teams and away_team not in actual_teams:
        continue
    result = match.FTR
    ######### Data to retrieve ##################
    # It can be score, number of goal ..
    # Here it's W/D/L result
    if result == "H":
        parcours[home_team].append("W")
        parcours[away_team].append("L")
    elif result == "A":
        parcours[away_team].append("W")
        parcours[home_team].append("L")
    else:
        parcours[away_team].append("D")
        parcours[home_team].append("D")
    #############################################
seasons[count] = {key : parcours[key] for key in actual_teams}
    

In [322]:
# Computing WDL streaks

def check_status(max,count,current_streak):
    if current_streak>max:
        count = 1
        max = current_streak
        current_streak = 0
    elif current_streak == max:
        current_streak = 0
        count+=1
    else:
        current_streak = 0
    return max,count,current_streak

def max_res_in_a_row(parcours,result): #In a single season

    max = 0
    count = 0
    current_streak = 0
    for match in parcours:
        if match == result:
            current_streak+=1
        else:
            max,count,current_streak = check_status(max,count,current_streak)
    max,count,current_streak = check_status(max,count,current_streak)
    return max,count


WDLstreaks = {team: {"W":[],"D":[],"L":[]} for team in actual_teams}
WDLstreaks_record = {team: {"W":(0,0),"D":(0,0),"L":(0,0)} for team in actual_teams}
issues = ["Win","Draw","Lose"]

for s in seasons:
    season = seasons[s]
    for team in season:
        path = season[team]
        for issue in issues:
            type = issue[0]
            season_record = max_res_in_a_row(path,type)
            WDLstreaks[team][type].append(season_record)
            if WDLstreaks_record[team][type][0] < season_record[0]:
                WDLstreaks_record[team][type] = season_record
            elif WDLstreaks_record[team][type][0] == season_record[0]:
                WDLstreaks_record[team][type] = (season_record[0],WDLstreaks_record[team][type][1]+season_record[1])
            

In [232]:
# Draw evolution of the streak by season

def streak_evolution(streak,team): #Season parameter in range btw 0:9, considering 0 in 2009-2010 and 9 in 2018-2019
    team_streak = streak[team]
    w,d,l = [],[],[]
    for ws in team_streak["W"]:
        w.append(ws[0])
    for ds in team_streak["D"]:
        d.append(ds[0])
    for ls in team_streak["L"]:
        l.append(ls[0])

    x = [f"{s}-{str(int(s)+1)}" for s in list(seasons.keys())]

    plt.plot(x,w,label = "Win streak")
    plt.plot(x,d,label = "Draw streak")
    plt.plot(x,l,label = "Lose streak")
    plt.legend()
    plt.xlabel("Season")
    plt.ylabel("Streak")
    plt.xticks(rotation=60)
    plt.title(f"Evolution of streak for {team} football team through the {len(x)} last seasons")
    plt.show()
    
    return w,d,l

for team in actual_teams:
    streak_evolution(WDLstreaks,team)

In [233]:
# Then we need to check the current streaks

def currentstreak(currentseason):
    current_streaks = {}
    for team in currentseason:
        parcours = currentseason[team]
        streak = 1
        type = parcours[-1]
        while parcours[-(1+streak)]==type:
            streak+=1
        current_streaks[team] = (type,streak)
    return current_streaks

current_streaks = currentstreak(seasons[2022])


In [310]:
# Proba que ça enchaîne sur le même résultat ?

def proba(team,result,serie): #Compute the percentage of each result knowing the current streak
    sum = 0
    count = 0
    for season in seasons:
        parcours = seasons[season][team]
        streak = 0
        for res in parcours:
            if streak == serie:
                if res == result:
                    sum+=1
                count+=1
            if res == result:
                streak+=1
            else:
                streak=0
    if count == 0:
        return (f"This serie never append in {team} last 12 seasons",0)
    return (sum/count,count)

In [321]:
result = pd.DataFrame(columns=["Team","Current Streak Type","Current Streak","Occurence in the last 12 seasons","Proba of pursuing the streak","Best streak","Occ best streak"])

probs = {}
for team in current_streaks:
    cur = current_streaks[team]
    p,total = proba(team,cur[0],cur[1])
    probs[team] = [p,cur[0],total]

    entry = pd.DataFrame.from_dict({
     "Team":[team],
     "Current Streak Type":[cur[0]],
     "Current Streak":[cur[1]],
     "Occurence in the last 12 seasons":[total],
     "Proba of pursuing the streak":[p],
     "Best streak":WDLstreaks_record[team][cur[0]][0],
     "Occ best streak":WDLstreaks_record[team][cur[0]][1]
    })
    result = pd.concat([result, entry], ignore_index=True)

result.style.set_caption("WDL Probability from streaks")


,Team,Current Streak Type,Current Streak,Occurence in the last 12 seasons,Proba of pursuing the streak,Best streak,Occ best streak
0,Paris SG,L,1,59,0.118644,4,1
1,Monaco,W,2,51,0.470588,12,1
2,Lens,W,1,48,0.250000,5,1
3,Montpellier,L,1,114,0.359649,5,2
4,Rennes,W,1,112,0.410714,5,4
5,Marseille,W,4,13,0.461538,8,1
6,Lorient,W,1,93,0.204301,6,1
7,Lille,D,1,98,0.275510,4,1
8,Lyon,L,1,93,0.215054,4,1
9,Nice,L,1,116,0.318966,7,1


### -0.5 halftime goal 

In [ ]:
# Compute parcours (Y : +0.5 HT goal, N : -0.5 HT goal)


In [ ]:
#Compute streaks (best streak)


In [ ]:
#Current streak


In [ ]:
# Proba and results


### +1.5 FT goals

### -3.5 FT goals

### Lay the draw

### -1.5 HT goals